In [16]:
import pandas as pd
import numpy as np
from src import data_cleaning as dc
from sklearn.neighbors import NearestNeighbors

In [2]:
# contains each players personal information
master_df = pd.read_csv('data/baseballdatabank-master/core/People.csv')

In [54]:
master_df[master_df['nameLast'] == 'Pujols']

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
14098,pujolal01,1980.0,1.0,16.0,D.R.,Distrito Nacional,Santo Domingo,NaN,NaN,NaN,...,Pujols,Jose Alberto,240.0,75.0,R,R,2001-04-02,2017-10-01,pujoa001,pujolal01
14099,pujollu01,1955.0,11.0,18.0,D.R.,Santiago,Santiago de los Caballeros,NaN,NaN,NaN,...,Pujols,Luis Bienvenido,175.0,74.0,R,R,1977-09-22,1985-05-22,pujol001,pujollu01


In [3]:
#create batting_df containing each players batting stats for each stint for each year
batting_df = pd.read_csv('data/baseballdatabank-master/core/Batting.csv')

In [4]:
#initial drop of useless data
dc.initial_drop(batting_df)

In [5]:
#combines players with multiple stints in one year into one line for entire year
batting_df_stints_combined = dc.combine_stints(batting_df)

In [6]:
#create df of players averages across their career
avg_df = dc.create_averages(batting_df_stints_combined)

In [7]:
#create fielding_df to grab positions from
fielding_df = pd.read_csv('data/baseballdatabank-master/core/Fielding.csv')

In [8]:
"""
fetch players' most played position from fielding_df and assign it in batting_df
this will drop players who are labeled as pitchers and those that did not have
a position assigned to them as they had very few ABs
"""
batting_pos_df = dc.map_position(batting_df_stints_combined, fielding_df)

In [96]:
player = 'goldspa01'
#get player's position
player_pos = batting_pos_df.loc[batting_pos_df['playerID'] == player, 'pos'].unique()[0]

#set masks
player_mask = batting_pos_df['playerID'] == player
pos_mask = batting_pos_df['pos'] == player_pos

#get service time
years_served = len(batting_pos_df[player_mask])

#get players that aren't 'player' but play same position
same_pos = batting_pos_df.loc[(~player_mask) & (pos_mask)]

#filter players of same position and their first n years equal to 'player's' service time
first_n_years = same_pos.groupby('playerID', as_index=False).head(years_served)

#filter players with at least as many years served as 'player'
first_n_years = players_to_compare_func(first_n_years)

In [95]:
def players_to_compare_func(df):
    value_counts = first_n_years.playerID.value_counts()
    players_to_compare = value_counts.loc[value_counts.values == 7].index
    return first_n_years[first_n_years['playerID'].isin(players_to_compare)]

In [99]:
same_pos.groupby('playerID', as_index=False).head()

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,pos
32,aaronto01,1962,141,334,54,77,20,2,8,38.0,6.0,0.0,41,58.0,0.0,0.0,4.0,3.0,10.0,1B
33,aaronto01,1963,72,135,6,27,6,1,1,15.0,0.0,3.0,11,27.0,1.0,0.0,3.0,2.0,7.0,1B
34,aaronto01,1965,8,16,1,3,0,0,0,1.0,0.0,0.0,1,2.0,0.0,0.0,0.0,0.0,1.0,1B
35,aaronto01,1968,98,283,21,69,10,3,1,25.0,3.0,4.0,21,37.0,1.0,0.0,2.0,1.0,9.0,1B
36,aaronto01,1969,49,60,13,15,2,0,1,5.0,0.0,1.0,6,6.0,0.0,0.0,0.0,0.0,1.0,1B
52,abadan01,2001,1,1,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1B
53,abadan01,2003,9,17,1,2,0,0,0,0.0,0.0,1.0,2,5.0,0.0,0.0,0.0,0.0,1.0,1B
54,abadan01,2006,5,3,0,0,0,0,0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,1B
63,abadijo01,1875,12,49,4,11,0,0,0,5.0,1.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0,1B
213,abreujo02,2014,145,556,80,176,35,2,36,107.0,3.0,1.0,51,131.0,15.0,11.0,0.0,4.0,14.0,1B


In [102]:
#get playerprediction class working for use